<a href="https://colab.research.google.com/github/Nico-Palacios/challenge-telecomX-parte2/blob/main/challenge_telecomX_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge TelecomX parte2

##Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

##Carga y Exploración de Datos

In [3]:
df = pd.read_csv('dataframe_tratado.csv')

In [4]:
df.sample(5)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
4470,6303-KFWSL,No,Female,0,No,No,29,Yes,Yes,DSL,...,No,No,No,No,One year,Yes,Electronic check,55.25,1620.20,1.841667
1198,1764-VUUMT,No,Male,0,No,Yes,66,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),19.95,1269.10,0.665000
3700,5206-XZZQI,No,Male,0,No,No,53,Yes,No,Fiber optic,...,No,Yes,No,No,Month-to-month,Yes,Mailed check,80.60,4348.10,2.686667
6559,9317-WZPGV,Yes,Female,1,No,No,2,Yes,No,Fiber optic,...,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),79.75,159.40,2.658333
3092,4431-EDMIQ,No,Female,0,Yes,Yes,7,Yes,No,DSL,...,Yes,Yes,Yes,No,Month-to-month,Yes,Electronic check,75.70,554.05,2.523333


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7043 non-null   object 
 3   SeniorCitizen     7043 non-null   int64  
 4   Partner           7043 non-null   object 
 5   Dependents        7043 non-null   object 
 6   tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   object 
 8   MultipleLines     7043 non-null   object 
 9   InternetService   7043 non-null   object 
 10  OnlineSecurity    7043 non-null   object 
 11  OnlineBackup      7043 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          7043 non-null   object 


In [6]:
df.describe()

,SeniorCitizen,tenure,Charges.Monthly,Charges.Total,Cuentas_Diarias
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304,2.158723
std,0.368612,24.559481,30.090047,2266.794470,1.003002
min,0.000000,0.000000,18.250000,0.000000,0.608333
25%,0.000000,9.000000,35.500000,398.550000,1.183333
50%,0.000000,29.000000,70.350000,1394.550000,2.345000
75%,0.000000,55.000000,89.850000,3786.600000,2.995000
max,1.000000,72.000000,118.750000,8684.800000,3.958333


###Eliminamos la columna customerID porque no aporta valor al análisis

In [7]:
# Eliminamos la columna 'customerID' del DataFrame de forma permanente
df.drop('customerID', axis=1, inplace=True)

In [8]:
# Aplica una función de mapeo a múltiples columnas categóricas para convertirtir todos los valores a yes - no
df[['Churn','Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']] = df[['Churn','Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']].applymap(lambda x: x if x in ['Yes','No'] else 'No')

/tmp/ipython-input-2924889110.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['Churn','Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']] = df[['Churn','Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']].applymap(lambda x: x if x in ['Yes','No'] else 'No')


In [19]:
# y_Ir contiene la variable que queremos predecir (target)
y_lr = df['Churn']

# x_Ir contiene todas las variables predictoras (features)
x_lr = df.drop(columns=['Churn'])

In [20]:
# Busca columnas con tipo de dato object, int64y float64
var_categoricas = x_lr.select_dtypes(include=['object']).columns.tolist()
var_numericas = x_lr.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [17]:
# Creamos un preprocesador de datos que aplica diferentes transformaciones a diferentes tipos de variables
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), var_numericas),('cat', OneHotEncoder(drop='first', sparse_output=False), var_categoricas)])